# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [71]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [72]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        if answer * self.f(x) > 1:
            der_w0 = -answer 
            der_w = -answer * x 
            return der_w, der_w0
        else:
            return np.zeros(len(x)), 0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            der_w, der_w0 = self.der_loss(x, y)
            # w update
            self.w -= step * (der_w + self.der_reg())
            
            # w_0 update
            self.w0 -= step * der_w0

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [73]:
from sklearn.metrics import classification_report

In [74]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[ 0.0288017   2.93735498  1.31037622  1.91084969  2.28187912  0.03059372
  0.32951538 -0.41683727 -0.34755783 -0.41654137  1.11653207  2.2603539
  1.87778243 -1.77890809 -2.06544893  0.57948947  1.41804704  2.31305495
  1.67545562  0.64871026] 0.0929666688756


In [75]:
predictions = model.predict(X_test)

In [76]:
print(predictions)

[0 0 0 ..., 0 1 1]


In [77]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [78]:
print(len(predictions), sum(predictions))

2000 1201


In [79]:
print(sum(predictions == y_test) / float(len(y_test)))

0.613


In [80]:
print(classification_report(y_train, model3.predict(X_train)))

             precision    recall  f1-score   support

          0       0.61      0.51      0.56      3997
          1       0.58      0.67      0.62      4003

avg / total       0.60      0.59      0.59      8000



## Сравнение качества с sklearn LinearSVC

In [81]:
from sklearn.svm import LinearSVC
svc = LinearSVC(C=5)
svc.fit(X_train, y_train);
print(svc.coef_, svc.intercept_)

[[ 0.00335287 -0.05295323 -0.00890857  0.03511552  0.00819404  0.13270897
   0.00198856  0.05243643 -0.08649099 -0.06468132  0.0571071   0.0704642
   0.14250531  0.02269189 -0.109833    0.03248952  0.05093768  0.08614315
  -0.06715961  0.04521074]] [-0.03303103]


In [82]:
print(classification_report(y_train, svc.predict(X_train)))

             precision    recall  f1-score   support

          0       0.79      0.74      0.76      3997
          1       0.75      0.80      0.78      4003

avg / total       0.77      0.77      0.77      8000

